In [1]:
import gdax
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib
import numpy as np

n = 4
public_client = gdax.PublicClient()

buy = False
sell = False
bidding = False
asking = False

PLN = 0 
BTC = 1
PLN_min = 1e-1
BTC_min = 1e-6

cci_thr = -60
mfi_thr = 0
cmo_thr = -120

# API_p = '557c4731915037e278adcb3aa96c029a'
# API_s = 'nt/cMZQ5rvxlDrK8qhw2wtqqZD16q3VyINlTpG6+Ba8pB5SsMWpS7VcyROPqZrFjAbD/dwEi0ucNsSCWGBzOsg=='

class Strategy:
    def __init__(self, param):
        self.time = datetime.datetime.now()
        self.c1 = param['c1']
        self.c2 = param['c2'] 
        self.c2_min = 0.001
        self.n = param['n']
        self.mfi_thr = param['mfi_thr']
        self.cci_thr = param['cci_thr']
        self.cmo_thr = param['cmo_thr']
        self.buy = False
        self.sell = False
        self.bidding = False
        self.asking = False
        self.queue = 0
        self.bid = 0
        self.ask = 0
        self.name = str(self.n) + '_' + str(self.mfi_thr) + '_' + str(self.cci_thr) + '_' + str(self.cmo_thr) + '_'

    def update(self, time, trades_ohlc, orderbook_l1):
        self.time = time
        average = trades_ohlc['close'].iloc[-1]
        equilibrium = self.c1/(self.c1+self.c2*average)
        
        window = trades_ohlc.iloc[-(self.n+1):]
    
        mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['size'].values, n)[-1]
        cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
        cmo = talib.CMO(window['close'].values, n)[-1]
        
        if cci > self.cci_thr and mfi > self.mfi_thr and cmo > self.cmo_thr:
            sell_new = True
        else:
            sell_new = False

        if cci < -self.cci_thr and mfi < 100 - self.mfi_thr and cmo < -self.cmo_thr:
            buy_new = True
        else:
            buy_new = False

        if(equilibrium > 0.01 and equilibrium < 0.99):
            if(self.buy and not buy_new and not sell_new):
                buy_new = True
            elif(self.sell and not sell_new and not buy_new):
                sell_new = True

        self.buy = buy_new
        self.sell = sell_new

        bids = orderbook_l1['bids'][0]
        asks = orderbook_l1['asks'][0]
        best_bid = float(bids[0])
        best_ask = float(asks[0])
        difference = round(best_ask - best_bid, 2)

        if self.bidding and self.bid != best_bid:
            if self.bid > best_bid + 0.01:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#             print('remove bid')
            self.bidding = False
        elif self.bidding:
            if self.queue - float(bids[1]) > 0:
                self.queue -= float(bids[1])
            if self.queue < 0:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#                 print('remove bid')
                self.bidding = False

        if(self.asking and self.ask != best_ask):
            if self.ask < best_ask - 0.01:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#             print('remove ask')
            self.asking = False
        elif self.asking:
            if self.queue - float(asks[1]) > 0:
                self.queue -= float(asks[1])
            if self.queue < 0:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#                 print('remove ask')
                self.asking = False

        if(self.buy and not self.bidding and self.c1 > self.c2_min*average):
            if(difference != 0.01):
#                 print('buy', best_bid + 0.01)    
                self.bid = best_bid + 0.01
                self.queue = self.c1/average
            else:
#                 print('buy', best_bid)
                self.bid = best_bid
                self.queue = float(bids[1])
            self.bidding = True


        if(self.sell and not self.asking and self.c2 > self.c2_min):
            if(difference != 0.01):
#                 print('sell', best_ask - 0.01)
                self.ask = best_ask - 0.01
                self.queue = self.c2
                queue_my = self.queue
            else:
#                 print('sell', best_ask)       
                self.ask = best_ask
                self.queue = float(asks[1])
            self.asking = True
            
        simulation = pd.DataFrame([[self.time, self.n, self.cci_thr, self.mfi_thr, self.cmo_thr, self.c1 + self.c2*average, self.c1/average + self.c2]], \
                            columns=['time', 'n', 'cci_thr', 'mfi_thr', 'cmo_thr', 'profit_c1', 'profit_c2'])
        simulation.to_csv('Simulation/GDAX_1900_02032018_300s_50n.csv', mode='a', header=0)
        return self.c1/average + self.c2, self.name


In [2]:
strategies = []


ns = np.linspace(10, 50, 11, dtype=np.int16)  #4
mfis = np.linspace(0, 90, 10) # 0
ccis = np.linspace(0, 90, 10) # 95
cmos = np.linspace(0, 90, 10) # 10

param= {}
c1 = 0
c2 = 1

for n in ns:
    for mfi_thr in mfis:
        for cci_thr in ccis:
            for cmo_thr in cmos:
                param['c1'] = c1
                param['c2'] = c2
                param['n'] = n
                param['mfi_thr'] = mfi_thr
                param['cci_thr'] = cci_thr
                param['cmo_thr'] = cmo_thr
                strategies.append(Strategy(param))
                
print(len(strategies))
                


# for i in range(1):
while True:    
    for attempt in range(3):
        try:
            historic_ohlc = public_client.get_product_historic_rates('BTC-EUR', granularity=300)
            orderbook_l1 = public_client.get_product_order_book('BTC-EUR', level=1)
        except:
            time.sleep(0.25)
        else:
            break
    else:
        print('Proba polaczenia nieudana')

    t = datetime.datetime.now()
        
    trades_ohlc = pd.DataFrame(data=historic_ohlc, columns=['date','low','high','open','close','size'])
    trades_ohlc.index = trades_ohlc['date'].values
    trades_ohlc = trades_ohlc.sort_index()
    
    profit_max = 0
    for strategy in strategies:
        profit, name = strategy.update(t, trades_ohlc, orderbook_l1)
        if profit > profit_max:
            profit_max = profit
            name_max = name 
    print(profit_max, name_max)
        
    print(datetime.datetime.now() - t)
    print(orderbook_l1)
    print(PLN, BTC)
    print()


11000
1.0 10_0.0_0.0_0.0_
0:02:40.187907
{'bids': [['8864.28', '3.22397186', 12]], 'asks': [['8864.29', '0.00246114', 1]], 'sequence': 3431126330}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.600374
{'bids': [['8868.95', '0.74712124', 11]], 'asks': [['8868.96', '1.52048044', 7]], 'sequence': 3431137761}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.938273
{'bids': [['8868.95', '0.30185554', 2]], 'asks': [['8868.96', '2.54265184', 13]], 'sequence': 3431145647}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.562941
{'bids': [['8849.71', '0.30263998', 2]], 'asks': [['8855', '0.17686652', 1]], 'sequence': 3431154045}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.783011
{'bids': [['8854.99', '0.0481922', 1]], 'asks': [['8855', '2.12072211', 10]], 'sequence': 3431161366}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.494907
{'bids': [['8854.99', '0.98140743', 4]], 'asks': [['8855', '1.14413045', 5]], 'sequence': 3431166194}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.256067
{'bids': [['8858.44', '0.69', 3]], 'asks': [['8858.45', '0.21', 2]], 'sequence': 3431170449}
0 

1.00303676657 50_50.0_80.0_0.0_
0:01:18.859724
{'bids': [['8851.52', '0.3230119', 4]], 'asks': [['8851.53', '2.47779088', 8]], 'sequence': 3431375646}
0 1

1.00309456209 50_50.0_80.0_0.0_
0:01:17.568290
{'bids': [['8848.44', '0.01', 1]], 'asks': [['8849.98', '0.09035269', 1]], 'sequence': 3431380589}
0 1

1.0034358047 50_50.0_80.0_0.0_
0:01:18.199968
{'bids': [['8848', '0.3530843', 4]], 'asks': [['8848.01', '2.27547437', 11]], 'sequence': 3431385331}
0 1

1.00376820082 50_50.0_80.0_0.0_
0:01:17.547233
{'bids': [['8845.07', '3.01985534', 12]], 'asks': [['8845.08', '1.21080186', 10]], 'sequence': 3431390522}
0 1

1.00376706598 50_50.0_80.0_0.0_
0:01:18.053578
{'bids': [['8845.07', '5.17218412', 22]], 'asks': [['8845.08', '0.78392973', 8]], 'sequence': 3431393661}
0 1

1.00376820082 50_50.0_80.0_0.0_
0:01:18.342348
{'bids': [['8845.07', '5.97829498', 25]], 'asks': [['8845.08', '0.43209583', 7]], 'sequence': 3431397361}
0 1

1.00354468839 50_50.0_80.0_0.0_
0:01:17.529186
{'bids': [['8847.1

1.0 10_0.0_0.0_0.0_
0:01:18.208994
{'bids': [['8921.59', '3.73354586', 15]], 'asks': [['8921.6', '0.64066735', 7]], 'sequence': 3431635653}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.641244
{'bids': [['8949.98', '1.0909', 5]], 'asks': [['8949.99', '0.75374785', 2]], 'sequence': 3431641001}
0 1

1.0 10_0.0_0.0_0.0_
0:01:19.319590
{'bids': [['8949.98', '0.83161696', 7]], 'asks': [['8949.99', '6.17553363', 16]], 'sequence': 3431644048}
0 1

1.0 10_0.0_0.0_0.0_
0:01:20.333082
{'bids': [['8949.98', '3.87299807', 11]], 'asks': [['8949.99', '5.90072935', 14]], 'sequence': 3431646879}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.078646
{'bids': [['8949.98', '3.21272926', 14]], 'asks': [['8949.99', '5.14145774', 13]], 'sequence': 3431649551}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.483725
{'bids': [['8949.98', '3.54354742', 17]], 'asks': [['8949.99', '2.30084657', 12]], 'sequence': 3431652100}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.682660
{'bids': [['8948.76', '0.10781', 2]], 'asks': [['8948.77', '0.31150193', 3]], 'sequence': 34

1.00021527997 50_0.0_0.0_30.0_
0:01:18.318840
{'bids': [['8965.06', '2.09074411', 10]], 'asks': [['8965.07', '1.90269483', 12]], 'sequence': 3431843334}
0 1

1.00021527997 50_0.0_0.0_30.0_
0:01:17.589982
{'bids': [['8965.06', '1.03992333', 6]], 'asks': [['8965.07', '3.358862', 14]], 'sequence': 3431847018}
0 1

1.00021639565 50_0.0_0.0_30.0_
0:01:18.502791
{'bids': [['8965.06', '1.34010161', 10]], 'asks': [['8965.35', '0.33343453', 1]], 'sequence': 3431850030}
0 1

1.0 10_0.0_0.0_0.0_
0:01:19.682346
{'bids': [['8967.49', '0.94965281', 5]], 'asks': [['8967.5', '1.07310233', 8]], 'sequence': 3431853247}
0 1

1.0 10_0.0_0.0_0.0_
0:01:19.524005
{'bids': [['8967.49', '1.26510804', 7]], 'asks': [['8967.5', '0.08458141', 3]], 'sequence': 3431856143}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.951487
{'bids': [['8967.49', '1.30705025', 5]], 'asks': [['8967.5', '2.53658465', 7]], 'sequence': 3431860766}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.285989
{'bids': [['8967.48', '0.1734536', 2]], 'asks': [['8967.49', '3

1.0 10_0.0_0.0_0.0_
0:01:16.921575
{'bids': [['8983.99', '4.04900259', 10]], 'asks': [['8984', '1.10500901', 3]], 'sequence': 3432026796}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.167227
{'bids': [['8983.99', '5.9018148', 13]], 'asks': [['8984', '2.25082544', 7]], 'sequence': 3432029737}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.911201
{'bids': [['8983.99', '5.02315079', 13]], 'asks': [['8984', '0.94760163', 6]], 'sequence': 3432032405}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.257964
{'bids': [['8983.99', '4.78370379', 16]], 'asks': [['8984', '1.80569205', 12]], 'sequence': 3432035859}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.915052
{'bids': [['8983.99', '5.44391106', 18]], 'asks': [['8984', '1.50804918', 10]], 'sequence': 3432039048}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.204322
{'bids': [['8983.99', '2.99999026', 14]], 'asks': [['8984', '1.29204918', 10]], 'sequence': 3432041551}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.432268
{'bids': [['8983.99', '2.03513556', 9]], 'asks': [['8984', '1.80788257', 15]], 'sequence': 3432044654}
0 

...

1.00205366922 50_0.0_0.0_60.0_
0:00:56.771904
{'bids': [['9130', '5.29731244', 16]], 'asks': [['9130.01', '1.71048551', 9]], 'sequence': 3433273165}
0 1

1.00205366922 50_0.0_0.0_60.0_
0:00:58.900500
{'bids': [['9130', '4.89445334', 10]], 'asks': [['9130.01', '0.6273461', 7]], 'sequence': 3433275348}
0 1

1.00205257168 50_0.0_0.0_60.0_
0:00:56.462153
{'bids': [['9130', '5.41492538', 15]], 'asks': [['9130.01', '0.21409546', 4]], 'sequence': 3433277051}
0 1

1.00205257168 50_0.0_0.0_60.0_
0:00:59.549939
{'bids': [['9133.23', '0.0262076', 1]], 'asks': [['9135', '0.0061116', 1]], 'sequence': 3433280400}
0 1

1.00150629612 50_0.0_0.0_60.0_
0:00:56.008151
{'bids': [['9139.33', '0.46743009', 6]], 'asks': [['9139.34', '0.15', 1]], 'sequence': 3433284335}
0 1

1.00102961483 50_0.0_0.0_60.0_
0:00:59.004307
{'bids': [['9143.99', '0.09441791', 3]], 'asks': [['9144', '3.0613778', 4]], 'sequence': 3433288667}
0 1

1.0 10_0.0_0.0_0.0_
0:00:55.615020
{'bids': [['9143.99', '2.63578243', 9]], 'asks': [[

1.00207232774 50_0.0_0.0_60.0_
0:00:58.705291
{'bids': [['9129.82', '0.83321981', 4]], 'asks': [['9129.83', '5.00937981', 15]], 'sequence': 3433457368}
0 1

1.00207342532 50_0.0_0.0_60.0_
0:00:57.582271
{'bids': [['9129.77', '1.37740819', 1]], 'asks': [['9129.78', '1.07929561', 4]], 'sequence': 3433459727}
0 1

1.00207891327 50_0.0_0.0_60.0_
0:00:58.526696
{'bids': [['9129.77', '1.88647515', 5]], 'asks': [['9129.78', '4.30447626', 13]], 'sequence': 3433462003}
0 1

1.00207891327 50_0.0_0.0_60.0_
0:00:57.659492
{'bids': [['9129.77', '2.43424261', 7]], 'asks': [['9129.78', '4.00624718', 10]], 'sequence': 3433464587}
0 1

1.00207891327 50_0.0_0.0_60.0_
0:00:58.325196
{'bids': [['9129.77', '0.73656693', 6]], 'asks': [['9129.78', '2.34453406', 9]], 'sequence': 3433468107}
0 1

1.00207891327 50_0.0_0.0_60.0_
0:00:58.072035
{'bids': [['9129.77', '5.94980221', 13]], 'asks': [['9129.78', '0.58700024', 2]], 'sequence': 3433471028}
0 1

1.00207781568 50_0.0_0.0_60.0_
0:00:58.567176
{'bids': [['91

1.0 10_0.0_0.0_0.0_
0:00:58.683315
{'bids': [['9159.47', '4.61368294', 10]], 'asks': [['9159.48', '9.65830859', 11]], 'sequence': 3433582945}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.790375
{'bids': [['9159.47', '6.5484405', 15]], 'asks': [['9159.48', '2.39259794', 6]], 'sequence': 3433584550}
0 1

1.0 10_0.0_0.0_0.0_
0:01:01.338999
{'bids': [['9159.47', '9.94822037', 19]], 'asks': [['9159.48', '2.0223119', 3]], 'sequence': 3433585939}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.463908
{'bids': [['9159.47', '13.99698368', 19]], 'asks': [['9159.48', '1.88840096', 6]], 'sequence': 3433587439}
0 1

1.0 10_0.0_0.0_0.0_
0:01:01.800200
{'bids': [['9159.47', '14.38235159', 21]], 'asks': [['9159.48', '0.93818243', 5]], 'sequence': 3433589036}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.165037
{'bids': [['9159.47', '14.46299903', 24]], 'asks': [['9159.48', '0.42762116', 6]], 'sequence': 3433590561}
0 1

1.0 10_0.0_0.0_0.0_
0:01:02.366096
{'bids': [['9159.99', '4.21606744', 13]], 'asks': [['9160', '0.48927893', 6]], 'sequenc

1.0 10_0.0_0.0_0.0_
0:00:59.849919
{'bids': [['9171.99', '5.36307943', 12]], 'asks': [['9172', '0.71498645', 6]], 'sequence': 3433674525}
0 1

1.0 10_0.0_0.0_0.0_
0:00:53.846718
{'bids': [['9172.09', '0.3643', 6]], 'asks': [['9172.1', '0.12721303', 3]], 'sequence': 3433675637}
0 1

1.0 10_0.0_0.0_0.0_
0:00:55.111196
{'bids': [['9172.09', '0.82357609', 10]], 'asks': [['9172.1', '5.47537205', 14]], 'sequence': 3433676623}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.362342
{'bids': [['9166.01', '1.28795119', 3]], 'asks': [['9166.02', '5.7580616', 14]], 'sequence': 3433679165}
0 1

1.0 10_0.0_0.0_0.0_
0:00:53.857142
{'bids': [['9159.47', '1.00827969', 3]], 'asks': [['9159.48', '2.80098172', 15]], 'sequence': 3433682426}
0 1

1.0 10_0.0_0.0_0.0_
0:00:55.704438
{'bids': [['9159.47', '0.53804266', 5]], 'asks': [['9159.48', '5.13153373', 19]], 'sequence': 3433684278}
0 1

1.0 10_0.0_0.0_0.0_
0:00:53.211043
{'bids': [['9159.47', '0.54622428', 3]], 'asks': [['9159.48', '4.82103456', 17]], 'sequence': 34336

KeyboardInterrupt: 